In [4]:
import os
import json
import shutil
import mimetypes
import base64
import time
from mistralai import Mistral
from dotenv import load_dotenv

# Chargement de la clé API depuis le fichier .env ou les variables d'environnement
# On a pu créer un environnement virtuel d'ou la clé est stocké dans un fichier
# Pour qu'on puisse y accéder directement, sans appel du contenu de texte
# Pour mesure de sécurité

load_dotenv()
api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key=api_key)

# Définition des dossiers utilisés dans le projet

# Dossier d'entrée : contient les fichiers à traiter
INPUT_DIR = "input"                    
# Dossier de sortie : stocke les résultats OCR au format JSON
OUTPUT_JSON_DIR = "output/json"
# Dossier d'archive : contient les fichiers déjà traités
PROCESSED_DIR = "processed"            

# Création des dossiers s'ils n'existent pas déjà
os.makedirs(OUTPUT_JSON_DIR, exist_ok=True)
os.makedirs(PROCESSED_DIR, exist_ok=True)

# Récupération de tous les fichiers présents dans le dossier d'entrée
files = [f for f in os.listdir(INPUT_DIR) if os.path.isfile(os.path.join(INPUT_DIR, f))]

if not files:
    print("Aucun fichier trouvé dans 'input/'")
else:
    for filename in files:
        file_path = os.path.join(INPUT_DIR, filename)
        base_name, ext = os.path.splitext(filename)
        ext = ext.lower()

        try:
            #  Traitement des fichiers PDF
            if ext == ".pdf":
                print(f"Traitement du PDF : {filename}")
                with open(file_path, "rb") as f:
                    uploaded_pdf = client.files.upload(
                        file={"file_name": filename, "content": f},
                        purpose="ocr"
                    )
                signed_url = client.files.get_signed_url(file_id=uploaded_pdf.id)

                # Exécution de l'OCR à partir du lien signé généré
                ocr_response = client.ocr.process(
                    model="mistral-ocr-latest",
                    document={"type": "document_url", "document_url": signed_url.url},
                    include_image_base64=True
                )

            #  Traitement des fichiers image (JPG, PNG...) 
            elif ext in [".png", ".jpg", ".jpeg"]:
                print(f"Traitement de l’image : {filename}")
                mime_type, _ = mimetypes.guess_type(file_path)
                with open(file_path, "rb") as f:
                    image_data = f.read()
                base64_encoded = base64.b64encode(image_data).decode("utf-8")
                image_url = f"data:{mime_type};base64,{base64_encoded}"

                # Exécution de l'OCR sur l'image encodée en base64
                ocr_response = client.ocr.process(
                    model="mistral-ocr-latest",
                    document={"type": "image_url", "image_url": image_url},
                    include_image_base64=True
                )

            else:
                # Cas où l'extension du fichier n'est pas supportée
                print(f"Format non pris en charge : {filename}")
                continue

            #  Enregistrement du résultat OCR au format JSON 
            with open(os.path.join(OUTPUT_JSON_DIR, base_name + ".json"), "w", encoding="utf-8") as f:
                json.dump(ocr_response.model_dump(), f, indent=2)

            #  Déplacement du fichier original dans le dossier d'archive 
            shutil.move(file_path, os.path.join(PROCESSED_DIR, filename))
            print(f"Fichier traité et archivé : {filename}")

            # Petite pause entre chaque fichier (précaution pour l'API)
            time.sleep(0.5)

        except Exception as e:
            # Affichage d'une erreur si le traitement échoue
            print(f"Erreur lors du traitement de {filename} : {e}")

Aucun fichier trouvé dans 'input/'
